In [15]:
# Koeffizienten: [ 4.01830776e+01  9.10906696e+01 -7.25100886e+00  5.24053555e+01
#  9.99527599e+01  3.89812761e+02  1.42753675e+02  6.80042819e+01
#  2.59061167e+02  7.53707701e+01 -1.51310585e+14 -1.51310585e+14
#  -1.51310585e+14 -1.51310585e+14 -1.51310585e+14 -1.51310585e+14
#  -1.51310585e+14 -1.51310585e+14 -1.51310585e+14 -1.51310585e+14]
# y-Achsenabschnitt: 151310585095729.97
# Mean Squared Error: 5583.109566266471
#
# FerienSH	4.01830776e+01 
# Feiertag	9.10906696e+01
# Weihnachtsmarkt	-7.25100886e+00
# Wochenende	 5.24053555e+01

# Warengruppe_1	 9.99527599e+01
# Warengruppe_2	 3.89812761e+02
# Warengruppe_3	 1.42753675e+02 
# Warengruppe_4	 6.80042819e+01

# Warengruppe_5	 2.59061167e+02
# Warengruppe_6	 7.53707701e+01
#   Jahreszeit_Temperatur_1Hoch  -1.51310585e+14
# 	Jahreszeit_Temperatur_1Mittel   -1.51310585e+14
# 	Jahreszeit_Temperatur_1Niedrig  -1.51310585e+14
# 	Jahreszeit_Temperatur_2Hoch  -1.51310585e+14
# 	Jahreszeit_Temperatur_2Mittel  -1.51310585e+14 
# 	Jahreszeit_Temperatur_3Hoch  -1.51310585e+14
# 	Jahreszeit_Temperatur_3Mittel  -1.51310585e+14
# 	Jahreszeit_Temperatur_3Niedrig   -1.51310585e+14 
# 	Jahreszeit_Temperatur_4Mittel   -1.51310585e+14
# 	Jahreszeit_Temperatur_4Niedrig   -1.51310585e+14


In [16]:
# import csv-files

import pandas as pd

pfad_1 = 'test.csv'
pfad_2 = 'fuf_v2.csv'
pfad_3 = 'weihnachtsmarkt_v2.csv'
pfad_4 = 'wetter_kaggle.csv'

# Einlesen der CSVs
df_1 = pd.read_csv(pfad_1, usecols=['id', 'Datum', 'Warengruppe'])
df_1['Datum'] = pd.to_datetime(df_1['Datum'])

df_2 = pd.read_csv(pfad_2, usecols=['Datum', 'FerienSH', 'Feiertag'])
df_2['Datum'] = pd.to_datetime(df_2['Datum'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
df_2['Datum'] = pd.to_datetime(df_2['Datum'])

df_3 = pd.read_csv(pfad_3, usecols=['Datum', 'Weihnachtsmarkt'])
df_3['Datum'] = pd.to_datetime(df_3['Datum'])

df_4 = pd.read_csv(pfad_4, usecols=['Datum', 'Temperatur'])
df_4['Datum'] = pd.to_datetime(df_4['Datum'])


dummies = pd.get_dummies(df_1['Warengruppe'], prefix='Warengruppe')
df_1 = pd.concat([df_1, dummies], axis=1)
# daten_2 = daten_2.drop('Warengruppe', axis=1)
df_1 = df_1.replace({True: 1, False: 0})

# Datensätze zusammenführen mit Inner Join
df = pd.merge(df_1, df_2, how='inner', on='Datum')
df = pd.merge(df, df_3, how='outer', on='Datum')
df = pd.merge(df, df_4, how='outer', on='Datum')

# Methode 1: Verwenden der len() Funktion
anzahl_datensaetze = len(df)
print(f"Anzahl der Datensätze: {anzahl_datensaetze}")


df['Weihnachtsmarkt'] = df['Weihnachtsmarkt'].fillna(0.0)
df['Weihnachtsmarkt'] = df['Weihnachtsmarkt'].astype(int)

df['Wochenende'] = df['Datum'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

df = df.assign(
    Jahreszeit_Temperatur_1Hoch=0,
    Jahreszeit_Temperatur_1Mittel=0,
    Jahreszeit_Temperatur_1Niedrig=0,
    Jahreszeit_Temperatur_2Hoch=0,
    Jahreszeit_Temperatur_2Mittel=0,
    Jahreszeit_Temperatur_3Hoch=0,
    Jahreszeit_Temperatur_3Mittel=0,
    Jahreszeit_Temperatur_3Niedrig=0,
    Jahreszeit_Temperatur_4Mittel=0,
    Jahreszeit_Temperatur_4Niedrig=0
)

df.loc[(df['Datum'].dt.month.isin([3, 4, 5])) & (df['Temperatur'] > 20), 'Jahreszeit_Temperatur_1Hoch'] = 1
df.loc[(df['Datum'].dt.month.isin([3, 4, 5])) & (df['Temperatur'] > 10) & (df['Temperatur'] < 20), 'Jahreszeit_Temperatur_1Mittel'] = 1
df.loc[(df['Datum'].dt.month.isin([3, 4, 5])) & (df['Temperatur'] < 10), 'Jahreszeit_Temperatur_1Niedrig'] = 1
df.loc[(df['Datum'].dt.month.isin([6, 7, 8])) & (df['Temperatur'] > 20), 'Jahreszeit_Temperatur_2Hoch'] = 1
df.loc[(df['Datum'].dt.month.isin([6, 7, 8])) & (df['Temperatur'] > 10) & (df['Temperatur'] < 20), 'Jahreszeit_Temperatur_2Mittel'] = 1
df.loc[(df['Datum'].dt.month.isin([9, 10, 11])) & (df['Temperatur'] > 20), 'Jahreszeit_Temperatur_3Hoch'] = 1
df.loc[(df['Datum'].dt.month.isin([9, 10, 11])) & (df['Temperatur'] > 10) & (df['Temperatur'] < 20), 'Jahreszeit_Temperatur_3Mittel'] = 1
df.loc[(df['Datum'].dt.month.isin([9, 10, 11])) & (df['Temperatur'] < 10), 'Jahreszeit_Temperatur_3Niedrig'] = 1
df.loc[(df['Datum'].dt.month.isin([12, 1, 2])) & (df['Temperatur'] > 10) & (df['Temperatur'] < 20), 'Jahreszeit_Temperatur_4Mittel'] = 1
df.loc[(df['Datum'].dt.month.isin([12, 1, 2])) & (df['Temperatur'] < 10), 'Jahreszeit_Temperatur_4Niedrig'] = 1





Anzahl der Datensätze: 1840


/tmp/ipykernel_10893/1605582744.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_1 = df_1.replace({True: 1, False: 0})


In [17]:
# df['Umsatz'] = 151310585095729.97 + df['FerienSH'] * 4.01830776e+01 + df['Feiertag'] * 9.10906696e+01 + df['Weihnachtsmarkt'] * (-7.25100886e+00)  + df['Wochenende'] * 5.24053555e+01 + df['Warengruppe_1'] * 9.99527599e+01 + df['Warengruppe_2'] * 3.89812761e+02   + df['Warengruppe_3'] * 1.42753675e+02   + df['Warengruppe_4'] * 6.80042819e+01 + df['Warengruppe_5'] * 2.59061167e+02   + df['Warengruppe_6'] * 7.53707701e+01  + df['Jahreszeit_Temperatur_1Hoch'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_1Mittel'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_1Niedrig'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_2Hoch'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_2Mittel'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_3Hoch'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_3Mittel'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_3Niedrig'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_4Mittel'] * (-1.51310585e+14) + df['Jahreszeit_Temperatur_4Niedrig'] * (-1.51310585e+14)

df['Umsatz'] =  df['FerienSH'] * 4.01830776e+01 + df['Feiertag'] * 9.10906696e+01 + df['Weihnachtsmarkt'] * (-7.25100886e+00)  + df['Wochenende'] * 5.24053555e+01 + df['Warengruppe_1'] * 9.99527599e+01 + df['Warengruppe_2'] * 3.89812761e+02   + df['Warengruppe_3'] * 1.42753675e+02   + df['Warengruppe_4'] * 6.80042819e+01 + df['Warengruppe_5'] * 2.59061167e+02   + df['Warengruppe_6'] * 7.53707701e+01

df.head(100)

df[['id','Umsatz']].to_csv('kaggle_export_file.csv', index=False)

